# Exercício final da disciplina de Sistemas de Recomendação, da pós em Inteligência Artificial da PUC Minas

## Recomendação Musical com base nas características das músicas

Aluno: **Pedo Henrique Rodrigues de Lima**

## 1. Introdução e importação dos datasets
Nesse projeto meu objetivo é recomendar músicas ao usuário, utilizando um sistema de recomendação híbrido, combinando uma recomendação baseada em conteúdo com uma abordagem colaborativa.

Os datasets foram retirados do site kaggle no seguinte link: https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks

Estou utilizando uma versão reduzida dos datasets disponibilizada no meu google drive.

Temos 3 datasets:

1. Dataset tracks, que contém informações sobre cada música, como: Duração, tom da música, "acusticabilidade", rítimo, ano de lançamento entre outras
2. Dataset artits, com informações sobre os artistas, como o gênero
3. Dataset user_tracks, que é um dataset criado por mim, com 7 usuários fictícios, e 5 músicas "curtidas" por cada um desses usuários

Importação dos datasets e das bibliotecas

In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial import distance
pd.options.display.max_columns = 200
pd.options.display.max_rows = 200

In [2]:
#!pip install scipy

In [3]:
from scipy.spatial import distance

In [4]:
#dataset contendo informações sobre as músicas, as informações de cada coluna, podem ser vistas no link do kaggle.
tracks = pd.read_csv('https://drive.google.com/uc?export=download&id=1C5DZ_yUGmqnJT_JpBRxLQuAs_smMIFAg', low_memory=False, encoding='utf-8')

In [5]:
artists = pd.read_csv('https://drive.google.com/uc?export=download&id=1UkKzFkRIg_6zcW5RktCd9XRdKfPnjgvT', low_memory=False, encoding='utf-8')
artists.drop(['Unnamed: 0',"Unnamed: 0.1","drop_genres"],axis = 1, inplace = True)

In [6]:
#dataset fictício, criado por mim, contendo o Id de um usuário fictíctio, e as músicas "curtidas" por ele.
user_tracks = pd.read_csv('https://drive.google.com/uc?export=download&id=1nQ1ZplDzOwIePUdTvjwam2eynlhPaa5b', low_memory=False,encoding='utf-8')

In [7]:
tracks.sample(5)

,Unnamed: 0,id_track,name_track,popularity_track,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,id_artists_first
215917,307037,4q7dBdsALA5BSunzryc104,I Could Be The One (Avicii Vs. Nicky Romero) -...,55,208316,0,"['Avicii', 'Nicky Romero']","['1vCWHaC5f2uS3yhpwWbIA6', '5ChF3i92IPZHduM7jN...",2012-01-01,0.509,0.790,6,-3.782,0,0.0374,0.3320,0.000067,0.316,0.638,127.946,4,1vCWHaC5f2uS3yhpwWbIA6
328258,482916,2PIQEyapdxvzPehxhjYbsg,Sarò Re,33,219320,0,"['Jim Cummings', 'Tullio Solenghi', 'Marco Gua...","['3OIPwMAZipDCLQwHmxS5EG', '1X5j4bH14OJi52Gz2v...",2003-01-01,0.663,0.642,9,-10.241,0,0.1180,0.1330,0.000000,0.110,0.381,122.037,4,3OIPwMAZipDCLQwHmxS5EG
276716,392028,4CLyoZC7TYZoalwFENy2Lv,ฮัก (Hug),19,234668,0,['Sukrit Wisetkaew'],['05ycwBAg6CYEweEbZgAYeT'],2009-04-09,0.869,0.785,7,-7.450,1,0.1310,0.0328,0.000001,0.270,0.832,130.019,4,05ycwBAg6CYEweEbZgAYeT
174057,258452,1lhMIksm9lnBulT9bNn8r1,Фотография 9х12,40,202250,0,['Irina Allegrova'],['1Zb7mgt9TRL16FhZa6qhXf'],1992,0.748,0.915,0,-2.631,0,0.0404,0.0530,0.000004,0.396,0.927,116.800,4,1Zb7mgt9TRL16FhZa6qhXf
70726,143058,159MiWYufUgQdrb3X5pgiX,How Can You Mind a Broken Heart,19,245156,0,['Pholhas'],['7vr15l4ulzvj0ef7x10jrO'],1985-11-11,0.271,0.215,4,-13.532,1,0.0373,0.8090,0.000000,0.101,0.108,201.158,3,7vr15l4ulzvj0ef7x10jrO


In [8]:
artists.sample(5)

,id_artist,followers,genres,name_artist,popularity_artist,track_release_year
24392,4WQXRya5np83C21wifjNp9,41394.0,['indie rock'],In The Valley Below,47,2014
14095,6caPJFLv1wesmM7gwK1ACy,757193.0,"['german house', 'german techno', 'minimal tec...",Boris Brejcha,66,2008
68,6n7nd5iceYpXVwcx8VPpxF,407124.0,"['classical', 'czech classical', 'late romanti...",AntonÃ­n DvoÅÃ¡k,63,1983
4552,7LQeFPitSkKhIskTd8knhP,10832.0,"['meditation', 'native american', 'new age']",Coyote Oldman,47,1993
3076,5VxhGHbcuvqTLRUWzBUN8L,341173.0,"['classic italian pop', 'italian adult pop']",Fiorella Mannoia,57,1984


In [9]:
user_tracks.sample(5)

,usuario,id_track,nome_musica
18,4,6Pq9MmkDQYZiiCDpxnvrf6,I Am The Walrus - Remastered 2009
30,7,7wCmS9TTVUcIhRalDYFgPy,Where Is My Mind? - Remastered
26,6,T1EaGm6b9eM7xBf4F1hlc,Maiden Voyage - Remastered
25,6,2DTHWr5NK1igQZxctpYTA9,Troupeau bleu
5,2,7kriFJLY2KOhw5en9iI2jb,Echoes


## 2. Tratamento dos datasets

Junção dos datasets de Músicas e de artistas no dataset **`tracks_artists`**

In [10]:
tracks_artists = pd.merge(left = tracks, right = artists, how = 'inner', 
                        left_on = 'id_artists_first',
                        right_on = 'id_artist')

Tratamento das colunas **`genres`**, **`artists`** e criação da coluna **`decade`**

In [11]:
tracks_artists['genres'] = tracks_artists['genres'].str.replace("'","",regex = False)
tracks_artists['genres'] = tracks_artists['genres'].str.replace("[","",regex = False)
tracks_artists['genres'] = tracks_artists['genres'].str.replace("]","",regex = False)
tracks_artists['genres'] = tracks_artists['genres'].str.replace(", ",",",regex = False)
tracks_artists['genres'] = tracks_artists['genres'].str.split(",")

In [12]:
tracks_artists['artists'] = tracks_artists['artists'].str.replace("'","",regex = False)
tracks_artists['artists'] = tracks_artists['artists'].str.replace("[","",regex = False)
tracks_artists['artists'] = tracks_artists['artists'].str.replace("]","",regex = False)

In [13]:
#cria uma coluna "década" em função do ano de publicação da música
tracks_artists['decade'] = (tracks_artists['track_release_year'].astype(str).str[-2]+"0").astype(int)

Transformação _one hot encoding_ das colunas categóricas

In [14]:
tracks_artists_dummies = pd.get_dummies(tracks_artists[['key','decade','time_signature']],columns = ['key','decade','time_signature'])

In [15]:
tracks_artists = pd.concat([tracks_artists,tracks_artists_dummies],axis = 1)
tracks_artists.set_index('id_track', drop=True, inplace=True)

Identificação dos generos existentes no dataset user_tracks das músicas curtidas pelos usuários

In [16]:
#generos únicos nas listas de músicas curtidas pelos usuarios (user_tracks)
tracks_ids = user_tracks['id_track'].values

all_genres = tracks_artists['genres'][tracks_artists.index.isin(tracks_ids)]

unique_genres = set([item for lista_generos in all_genres for item in lista_generos])
unique_genres = [genero.replace('"',"") for genero in unique_genres]
unique_genres = [re.sub(r'^ ',"",genero) for genero in unique_genres]

In [17]:
print(unique_genres)

['lo-fi', 'nu gaze', 'garage rock', 'progressive rock', 'shoegaze', 'jazz funk', 'alternative pop', 'newcastle nsw indie', 'hard rock', 'brazilian boogie', 'zolo', 'nova mpb', 'dance-punk', 'brazilian jazz', 'lo-fi indie', 'edmonton indie', 'hard bop', 'acid rock', 'pagode', 'samba-rock', 'double drumming', 'brazilian rock', 'jazz trumpet', 'indie pop', 'classic rock', 'mpb', 'melancholia', 'musica popular mineira', 'bossa nova', 'jazz', 'psicodelia brasileira', 'samba de roda', 'jazz fusion', 'new wave', 'alternative metal', 'modern rock', 'canadian singer-songwriter', 'boston rock', 'art rock', 'rock', 'british invasion', 'album rock', 'indie rock', 'country rock', 'roots rock', 'indie garage rock', 'oxford indie', 'experimental rock', 'samba', 'post-punk', 'anti-folk', 'krautrock', 'neo-psychedelic', 'classic canadian rock', 'merseybeat', 'glam rock', 'brazilian soul', 'grunge', 'tropicalia', 'cool jazz', 'freak folk', 'pop rock', 'australian psych', 'australian indie', 'beatlesque'

Inclusão de uma coluna com valores 0 ou 1 para cada gênero curtido pelos usuários (não vou usar todos os gêneros do dataset artists, pois do contrário teríamos milhares de colunas para os gêneros). 

In [18]:
for g in unique_genres:
    tracks_artists[g] = np.zeros(len(tracks_artists))

for index, row in tracks_artists.iterrows():
    for genre in row['genres']:
        if genre in unique_genres:
            tracks_artists.at[index, genre] = 1
        else:
            pass

Criação de um dataset **normalizado**, contendo apenas as colunas com características das músicas.

In [19]:
trk_chars = tracks_artists.select_dtypes(include = 'number').drop(['Unnamed: 0', 'popularity_track', 'explicit',
       'key','time_signature','followers', 'popularity_artist',
       'track_release_year', 'decade'], axis = 1)

indices = trk_chars.index
colunas = trk_chars.columns

trk_chars_norm = MinMaxScaler().fit_transform(trk_chars)

trk_chars_norm = pd.DataFrame(trk_chars_norm, index = indices, columns = colunas)

In [20]:
trk_chars_norm.sample(5)

,duration_ms,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11,decade_0,decade_10,decade_20,decade_60,decade_70,decade_80,decade_90,time_signature_0,time_signature_1,time_signature_3,time_signature_4,time_signature_5,lo-fi,nu gaze,garage rock,progressive rock,shoegaze,jazz funk,alternative pop,newcastle nsw indie,hard rock,brazilian boogie,zolo,nova mpb,dance-punk,brazilian jazz,lo-fi indie,edmonton indie,hard bop,acid rock,pagode,samba-rock,double drumming,brazilian rock,jazz trumpet,indie pop,classic rock,mpb,melancholia,musica popular mineira,bossa nova,jazz,psicodelia brasileira,samba de roda,jazz fusion,new wave,alternative metal,modern rock,canadian singer-songwriter,boston rock,art rock,rock,british invasion,album rock,indie rock,country rock,roots rock,indie garage rock,oxford indie,experimental rock,samba,post-punk,anti-folk,krautrock,neo-psychedelic,classic canadian rock,merseybeat,glam rock,brazilian soul,grunge,tropicalia,cool jazz,freak folk,pop rock,australian psych,australian indie,beatlesque,psychedelic rock,instrumental rock,jazz rock,experimental,space rock,southern rock,alternative rock,mellow gold,dream pop,singer-songwriter,symphonic rock,swamp rock,brazilian indie,manguebeat,bay area indie,permanent wave
id_track,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1YxZFhM4JapByFt0a3XRB7,0.034966,0.534413,0.112,0.746847,1.0,0.042312,0.921687,0.000000,0.1060,0.314,0.708099,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1b7LMtXCXGc2EwOIplI35z,0.038378,0.804656,0.564,0.776826,1.0,0.040454,0.093775,0.000531,0.0631,0.961,0.509907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2R1dnEOe2EfrvD88KBwZBm,0.027275,0.289474,0.160,0.762425,1.0,0.031992,0.862450,0.000000,0.3440,0.187,0.366363,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5FHagG20bbo7bSqFJNvtJr,0.033588,0.447368,0.884,0.853994,0.0,0.038700,0.149598,0.001630,0.0545,0.934,0.661316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0IL79jSTiVN3WNLegdFhIl,0.043650,0.591093,0.635,0.857508,1.0,0.033333,0.404618,0.000000,0.0862,0.225,0.591843,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [21]:
def get_user_artists(user_id):
    '''Recebe o user_id (int) e retorna lista com os artistas das músicas curtidas pelo usuário'''
    return tracks_artists['artists'][tracks_artists.index.isin(user_tracks['id_track'][user_tracks['usuario']==user_id])].values

Exibindo os artistas curtidos por cada usuário

In [22]:
for user in range(1,8):
    print(user,get_user_artists(user),end = '\n\n')

1 ['Milton Nascimento' 'Arthur Verocai' 'Lô Borges' 'Lô Borges'
 'Beto Guedes']

2 ['Pink Floyd' 'Pink Floyd' 'King Crimson' 'Emerson, Lake & Palmer' 'CAN']

3 ['Jorge Ben Jor' 'Caetano Veloso' 'Tim Maia' 'Novos Baianos' 'Os Mutantes']

4 ['The Beatles' 'The Doors' 'David Bowie' 'Creedence Clearwater Revival'
 'Neil Young, Crazy Horse']

5 ['The Brian Jonestown Massacre' 'Mac DeMarco' 'Tame Impala' 'Vacations'
 'King Gizzard & The Lizard Wizard']

6 ['Miles Davis' 'Cortex']

7 ['Pixies' 'Nirvana' 'The Smashing Pumpkins' 'Radiohead' 'Pavement']



## 3. Recomendação baseada em conteúdo

In [23]:
def get_user_df(user_id):
    '''Recebe o user_id (int) e retorna o dataframe normalizado com as músicas curtidas pelo usuário'''
    tracks_ids = user_tracks['id_track'][user_tracks['usuario'] == user_id].values
    trk_chars_norm_user = trk_chars_norm[trk_chars_norm.index.isin(tracks_ids)]
    trk_chars_norm_user = tracks_artists[['name_track','artists']].join(trk_chars_norm_user,how = 'right')
    return trk_chars_norm_user

In [24]:
def get_user_profile(user_id):
    '''Recebe o user_id (int) e retora o user profile, que é a mediana das features das músicas curtidas'''
    user_df = get_user_df(user_id).iloc[:,2:]
    user_profile = user_df.median()
    return user_profile

In [25]:
def get_user_recomendations_content(user_id, top = 10, new_artists = False):
    '''Recebe o user_id (int) e retorna as top 10 recomendações para o usuário com base em conteúdo
    Se new_artists = True, retorna apenas bandas que não estavam originalmente nas músicas curtidas pelo usuario'''
    user_profile = get_user_profile(user_id) #perfil do usuário
    trk_chars_norm_comparison = trk_chars_norm.copy() #dataset completo com característica das músicas característidas
    trk_chars_norm_comparison['user_score'] = np.nan
    for i, row in trk_chars_norm_comparison.iterrows():
        row['user_score'] = distance.euclidean(row[0:-1],user_profile) #calcula a similidade com o perfil do user, usando distância euclidiana
    
    artistas_user = get_user_artists(user_id)
    
    #ordena as top 10 menores distâncias
    top_artists = pd.concat([trk_chars_norm_comparison,tracks_artists[['name_track','artists']]],axis = 1)
    top_artists = top_artists.sort_values(by = 'user_score')
    
    if new_artists:
        top_artists = top_artists[~top_artists['artists'].isin(artistas_user)].head(top)
    else:
        top_artists = top_artists.head(top)
     
    top_artists = top_artists[['name_track','artists']]
    
    return top_artists

Recomendações para o usuário 7

In [26]:
get_user_recomendations_content(7, top = 5)

,name_track,artists
id_track,,
2bbhaUgu8FbGOmO22KyaIh,Ashtray Heart,Placebo
3HJgJWqeTKX3gwoZrn4R56,Black-Eyed,Placebo
4JuNnJWmvZiD2e0LQSjuIh,Only Happy When It Rains,Garbage
3DwzYRgOgojl0EcdOaJLYr,Tahitian Moon,Porno for Pyros
6p6Xq0t9XGkVg04cXWdx8v,Only Happy When It Rains - 2015 - Remaster,Garbage


Recomendações para o usuário 1, **considerando apenas artistas novos**

In [27]:
get_user_recomendations_content(1, new_artists = True)

,name_track,artists
id_track,,
6Bb0x4G5bl7PoZ3KUConGe,Clube Da Esquina Nº 2,"Lô Borges, Solange Borges"
0EUTShHUeDpQgPmaE4pQOb,Vento De Maio,"Lô Borges, Solange Borges"
2Jswl0dxLim39ZhbERefuX,Manuel O Audaz,"Lô Borges, Toninho Horta, Pat Metheny"
5T68pG61pSRG0sE20tjhfK,Manuel O Audaz,"Lô Borges, Toninho Horta, Pat Metheny"
1JYpbkS0Lei8gXlQ1beWas,Deixa,Vinícius de Moraes
08oqG2aZTJndXXwGOAth9n,Vitoriosa - Ao Vivo,Ivan Lins
0BAI8k3qquS5LlRAV2MWs9,Pesadelo,MPB4
2XrMKEPMNDiFVK3j7Zc7B2,Fulô do Dia,Geraldo Azevedo
3vJJqxkNLVHibD3gRttOHP,Vitoriosa,Ivan Lins


## 4. Recomendação híbrida monolítica, combinando conteúdo e o vizinho mais próximo

Nessa recomendação farei o seguinte:

1. Encontrarei as top 20 músicas mais parecidas com as músicas curtidas pelo usuário
2. Encontrarei o vizinho mais próximo do usuário
3. Com base no user_profile do **vizinho** mais próximo, recomendarei as top N músicas no dubset gerado no passo 1

In [28]:
def get_all_user_profiles(df = user_tracks):
    '''Cria o user profile de todos os 7 usuarios'''
    all_user_profiles = pd.DataFrame(columns = (get_user_profile(1).index))
    for u in user_tracks['usuario'].unique():
        user_profile = pd.DataFrame(data = get_user_profile(u).values.reshape(1,len(get_user_profile(u))), index = [u], columns =get_user_profile(u).index)
        all_user_profiles = pd.concat([all_user_profiles,user_profile])
    
    return all_user_profiles

In [29]:
get_all_user_profiles()

,duration_ms,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11,decade_0,decade_10,decade_20,decade_60,decade_70,decade_80,decade_90,time_signature_0,time_signature_1,time_signature_3,time_signature_4,time_signature_5,lo-fi,nu gaze,garage rock,progressive rock,shoegaze,jazz funk,alternative pop,newcastle nsw indie,hard rock,brazilian boogie,zolo,nova mpb,dance-punk,brazilian jazz,lo-fi indie,edmonton indie,hard bop,acid rock,pagode,samba-rock,double drumming,brazilian rock,jazz trumpet,indie pop,classic rock,mpb,melancholia,musica popular mineira,bossa nova,jazz,psicodelia brasileira,samba de roda,jazz fusion,new wave,alternative metal,modern rock,canadian singer-songwriter,boston rock,art rock,rock,british invasion,album rock,indie rock,country rock,roots rock,indie garage rock,oxford indie,experimental rock,samba,post-punk,anti-folk,krautrock,neo-psychedelic,classic canadian rock,merseybeat,glam rock,brazilian soul,grunge,tropicalia,cool jazz,freak folk,pop rock,australian psych,australian indie,beatlesque,psychedelic rock,instrumental rock,jazz rock,experimental,space rock,southern rock,alternative rock,mellow gold,dream pop,singer-songwriter,symphonic rock,swamp rock,brazilian indie,manguebeat,bay area indie,permanent wave
1,0.044199,0.453441,0.3820,0.721150,1.0,0.031992,0.441767,0.000015,0.1530,0.503,0.578571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.111981,0.288462,0.3220,0.681162,0.0,0.039009,0.367470,0.651000,0.1070,0.297,0.564904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.041201,0.596154,0.5810,0.809504,1.0,0.091950,0.462851,0.000001,0.0830,0.773,0.428329,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.052824,0.470648,0.5560,0.813049,1.0,0.038596,0.085643,0.000359,0.1520,0.535,0.511764,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.044795,0.533401,0.6360,0.843232,0.0,0.051703,0.060141,0.013200,0.1310,0.594,0.388363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.065485,0.442308,0.5975,0.781423,0.5,0.036068,0.395080,0.359214,0.1769,0.627,0.6397

In [30]:
def get_closest_user(user_id,df = user_tracks):
    '''Recebe o user_id (int) e retorna o id do usuário com gostos mais parecidos'''
    all_user_profiles = get_all_user_profiles(df)
    user_profile = all_user_profiles.loc[user_id,:]
    orther_user_profiles = all_user_profiles.drop(user_id, axis = 0)
    
    distance_dict = {}
    
    for u in orther_user_profiles.index:
        other_user_profile = all_user_profiles.loc[u,:]
        distance_users = distance.euclidean(user_profile,other_user_profile)
        distance_dict[u] = distance_users
     
    closest_user = min(distance_dict, key=distance_dict.get)
    
    return closest_user

In [31]:
def get_hybrid_recomendations_content(user_id, top = 10, new_artists = False):
    '''Recebe o user_id (int) e retorna as top recomendações para o usuário considerando também o perfil do usuário mais próximo a ele'''
    
    # Top recomendações para o primeiro usuario
    user_profile = get_user_profile(user_id) #perfil do usuário
    trk_chars_norm_comparison = trk_chars_norm.copy() #dataset completo com característica das músicas característidas
    trk_chars_norm_comparison['user_score'] = np.nan
    for i, row in trk_chars_norm_comparison.iterrows():
        row['user_score'] = distance.euclidean(row[0:-1],user_profile) #calcula a similidade com o perfil do user, usando distância euclidiana
    
   
    top_artists = trk_chars_norm_comparison.sort_values(by = 'user_score').join(tracks_artists[['artists','name_track']])

    artistas_user = get_user_artists(user_id)
    
    #Escolha se o o usuário quer só artistas novos ou não
    if new_artists:
        top_artists = top_artists[~top_artists['artists'].isin(artistas_user)].head(20)
    else:
        top_artists = top_artists.head(20)
           
        
    # Usuario mais próximo
    neighbor = get_closest_user(user_id)
    
    # Cálculo das distâncias usando as recomendações do primeiro usuario e o profile do usuario mais próximo
    neighbor_profile = get_user_profile(neighbor) #perfil do vizinho mais próximo
    first_user_rec_comparison = top_artists.copy() #dataset com as recomendações do primeiro usuario
    first_user_rec_comparison['user_score'] = np.nan
    first_user_rec_comparison =  first_user_rec_comparison.drop(['name_track','artists'],axis = 1)
    
    for i, row in first_user_rec_comparison.iterrows():
        row['user_score'] = distance.euclidean(row[0:-1],neighbor_profile) #calcula a similidade com o perfil do user, usando distância euclidiana
   
  
    #ordena as top menores distâncias
    top_hybrid = first_user_rec_comparison.sort_values(by = 'user_score').head(top)
    top_hybrid = first_user_rec_comparison.sort_values(by = 'user_score').head(top).join(tracks_artists[['name_track','artists']])
    top_hybrid = tracks_artists[['name_track','artists']][tracks_artists.index.isin(top_hybrid.index)]
    return top_hybrid

Recomendações **híbridas** para o usuário 4

In [32]:
get_hybrid_recomendations_content(4)

,name_track,artists
id_track,,
4XOMLnCpUmajrwd07JjIqf,Please Let Me Wonder (Stereo),The Beach Boys
0nc4t5n6r5karpDkS6F8ab,Transcendental Meditation - Remastered 2001,The Beach Boys
04WYXNdpDSewhyWFihqYI8,We Gotta Get Out Of This Place - US Single Ver...,The Animals
4CyqGx74ElL4WC5TaEbx0x,Not to Touch the Earth,The Doors
7gf3Brl7VbnMHn07BVKUoq,Gloria - Live,The Doors
2nCRjFVNNfpedJevLrMBLI,Into the Fire - 1995 Remaster,Deep Purple
0oCe272nVMTE2vYAbjHlM3,Holy Man - Quadrophonic Mix;Stereo;2009 Digita...,Deep Purple
6aqWPEhtxJaPbSabre2fdx,Woman from Tokyo,Deep Purple
3f20tlp0TMHRwE84Ung7RG,Into The Fire,Deep Purple


Recomendações **baseadas somente em conteúdo** para o usuário 4

In [33]:
get_user_recomendations_content(4)

,name_track,artists
id_track,,
6DRUtMPBop3V0lz47i65JE,Please Let Me Wonder (Mono),The Beach Boys
0nc4t5n6r5karpDkS6F8ab,Transcendental Meditation - Remastered 2001,The Beach Boys
04WYXNdpDSewhyWFihqYI8,We Gotta Get Out Of This Place - US Single Ver...,The Animals
4CyqGx74ElL4WC5TaEbx0x,Not to Touch the Earth,The Doors
0vZ4hMzKYEAMlptfxXumWf,Marcella - Remastered 2000,The Beach Boys
1hcMfYTsRTC4hIKbfosxjz,I Can Hear Music,The Beach Boys
6BzZvQJsXi68wSvFTHU9bC,Slip On Through - Remastered 2009,The Beach Boys
4XOMLnCpUmajrwd07JjIqf,Please Let Me Wonder (Stereo),The Beach Boys
7CRJ8BC8ZdpX6ep15N0rp3,Woman From Tokyo - 2000 Remix,Deep Purple
